# Install Earth Engine Python API

Install the Earth Engine Python API and geehydro. The geehydro Python package builds on the folium package and implements several methods for displaying Earth Engine data layers, such as Map.addLayer(), Map.setCenter(), Map.centerObject(), and Map.setOptions(). The magic command %%capture can be used to hide output from a specific cell. 

In [1]:
# %%capture
!pip install earthengine-api
!pip install geehydro

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10143 sha256=7c1124deefae472f5154035a16225a7452faa877fab8cff36cc9b3c1700a7ca4
  Stored in directory: /home/chengyu/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro


In [3]:
## Import the Earth Engine library.Also import the folium and geehydro libraries. 
import ee
import folium
import geehydro

## Authenticate

Import the Earth Engine API and run the `ee.Authenticate` function to authenticate your access to Earth Engine servers. Upon running the cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line ee.Authenticate() if you are running this notebook for the first time or if you are getting an authentication error.

In [6]:
#ee.Authenticate()
ee.Initialize()

## Create an interactive map

This step creates an interactive map using folium. The default basemap is the OpenStreetMap. Additional basemaps can be added using the Map.setOptions() function. The optional basemaps can be ROADMAP, SATELLITE, HYBRID, TERRAIN, or ESRI.

In [4]:
Map = folium.Map(location=[14.09, -59.9358], zoom_start=3) #Brazil latitude, longitude
Map.setOptions('HYBRID')

## Calculate NDVI 

In [7]:
#Load countries data
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");     ##import one database
Brazil = countries.filter(ee.Filter.eq("country_na","Brazil")); ## filter brazil only

# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = landsat1999.select('B4').subtract(landsat1999.select('B3')) \
               .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

# Compute the multi-band difference image.
diff = landsat2008.subtract(landsat1999)
Map.addLayer(diff.clip(Brazil),
             {'bands': ['B4', 'B3', 'B2'], 'min': -32, 'max': 32},
             'difference')

## Display

In [8]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map